In [ ]:
import os
import time
import logging
from telegram import Update
from telegram.ext import (
    ApplicationBuilder, CommandHandler, MessageHandler,
    filters, ContextTypes, ConversationHandler
)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from PyPDF2 import PdfReader
import google.generativeai as genai

# === НАСТРОЙКИ ===
TELEGRAM_BOT_TOKEN = "ВАШ_ТОКЕН_ТЕЛЕГРАМ_БОТА"
GEMINI_API_KEY = "ВАШ_GEMINI_API_КЛЮЧ"

# === ПОДГОТОВКА ===
genai.configure(api_key=GEMINI_API_KEY)
download_dir = os.path.abspath("downloads/programmes")
os.makedirs(download_dir, exist_ok=True)
WAITING_FOR_LINKS, WAITING_FOR_QUESTIONS = range(2)

# === ИНИЦИАЛИЗАЦИЯ SELENIUM ===
def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "profile.default_content_setting_values.automatic_downloads": 1,
    }
    options.add_experimental_option("prefs", prefs)
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# === СКАЧИВАНИЕ PDF И ТЕКСТ ===
def download_pdf_and_get_extra_text(driver, url):
    driver.get(url)
    wait = WebDriverWait(driver, 15)
    button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Скачать учебный план')]")))
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", button)

    timeout = 15
    file_path = None
    for _ in range(timeout * 2):
        files = os.listdir(download_dir)
        pdf_files = [f for f in files if f.endswith(".pdf")]
        if pdf_files:
            file_path = os.path.join(download_dir, pdf_files[0])
            break
        time.sleep(0.5)

    pdf_text = ""
    if file_path and os.path.exists(file_path):
        with open(file_path, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                pdf_text += page.extract_text() or ""

    extra_text = ""
    try:
        extra_element = driver.find_element(By.XPATH, "/html/body/div/div/main/div[3]/div[1]/div/div[1]/div/span[2]")
        extra_text = extra_element.text
    except Exception:
        pass

    return pdf_text, extra_text, file_path

# === ОБРАБОТКА GEMINI ===
def ask_gemini(user_question, plan1, plan2, extra1, extra2):
    model = genai.GenerativeModel("gemini-pro")
    prompt = f"""
Ты — помощник, отвечающий кратко и ясно на вопросы об учебных программах. Пользователь задал вопрос:

"{user_question}"

Вот дополнительная информация:

Доп. текст программы 1: {extra1}
Доп. текст программы 2: {extra2}

Учебный план 1:
{plan1}

Учебный план 2:
{plan2}

Дай короткий и точный ответ на вопрос пользователя.
"""
    response = model.generate_content(prompt)
    return response.text.strip()

# === ХЭНДЛЕРЫ БОТА ===
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Привет! Пришли 2 ссылки на программы через пробел.")
    return WAITING_FOR_LINKS

async def handle_links(update: Update, context: ContextTypes.DEFAULT_TYPE):
    urls = update.message.text.strip().split()
    if len(urls) != 2:
        await update.message.reply_text("Нужно отправить ровно 2 ссылки через пробел.")
        return WAITING_FOR_LINKS

    for f in os.listdir(download_dir):
        if f.endswith(".pdf"):
            os.remove(os.path.join(download_dir, f))

    driver = init_driver()
    try:
        pdf1, extra1, _ = download_pdf_and_get_extra_text(driver, urls[0])
        for f in os.listdir(download_dir):
            if f.endswith(".pdf"):
                os.remove(os.path.join(download_dir, f))
        pdf2, extra2, _ = download_pdf_and_get_extra_text(driver, urls[1])
    finally:
        driver.quit()

    context.user_data["pdf1"] = pdf1
    context.user_data["pdf2"] = pdf2
    context.user_data["extra1"] = extra1
    context.user_data["extra2"] = extra2

    await update.message.reply_text("Файлы загружены. Теперь задай вопрос по этим программам.")
    return WAITING_FOR_QUESTIONS

async def handle_question(update: Update, context: ContextTypes.DEFAULT_TYPE):
    data = context.user_data
    question = update.message.text.strip()

    if not all(k in data for k in ["pdf1", "pdf2", "extra1", "extra2"]):
        await update.message.reply_text("Сначала отправь 2 ссылки с программами.")
        return WAITING_FOR_LINKS

    await update.message.reply_text("Обрабатываю запрос...")

    reply = ask_gemini(question, data["pdf1"], data["pdf2"], data["extra1"], data["extra2"])
    await update.message.reply_text(reply)
    return WAITING_FOR_QUESTIONS

async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Диалог завершён.")
    return ConversationHandler.END

# === MAIN ===
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    app = ApplicationBuilder().token(TELEGRAM_BOT_TOKEN).build()

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler("start", start)],
        states={
            WAITING_FOR_LINKS: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_links)],
            WAITING_FOR_QUESTIONS: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_question)],
        },
        fallbacks=[CommandHandler("cancel", cancel)],
    )

    app.add_handler(conv_handler)
    print("Бот запущен...")
    app.run_polling()
